# Clasificador de generos musicales

Como ejemplo del uso de modelos que funcionan en audio, haremos un clasificador de géneros musicales. Para hacer esto, utilizaremos el conjunto de datos GTZAN, un conjunto de datos de 1000 muestras de audio etiquetadas con el género de la música.

## Instalación de librerías
Para ejecutar este cuaderno, necesitaremos instalar las siguientes librerías:

In [ ]:
%pip install transformers datasets librosa soundfile torch accelerate evaluate

import os
os.environ["WANDB_DISABLED"] = "true"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

## Cargamos el dataset

In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all", trust_remote_code=True)
gtzan

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

gtzan.py:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

genres.tar.gz:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

Como podemos ver, el conjunto de datos consta de 999 muestras de audio etiquetadas con el género de la música.

Los audios están en formato de 22050 Hz, para procesarlos con el modelo, necesitaremos convertirlos en un formato que pueda procesar el modelo (generalmente 16 kHz). Haremos esto con la clase `Audio` de la librería `datasets`.

In [3]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=16000))

## Creación del conjunto de datos `test`

Para evaluar el modelo necesitaremos un conjunto de datos de prueba. Para hacer esto, dividiremos el conjunto de datos en dos partes, uno para entrenar el modelo y otro para evaluarlo.

In [4]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

Una vez que el conjunto de datos se ha separado en dos partes, el conjunto de datos de prueba contendrá 100 muestras de audio.

A continuación, mostraremos una muestra del conjunto de datos de prueba.

In [5]:
gtzan['train'][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/3b204381d6c029312e4f9c569c6b1130af3041dd36ca38ca53d4e20f585e39c6/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3b204381d6c029312e4f9c569c6b1130af3041dd36ca38ca53d4e20f585e39c6/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

De cada muestra del conjunto de datos de prueba podemos ver que tenemos los siguientes datos:
- `audio`: la ruta en el archivo de audio.
- `array`: audio en formato de matriz.El valor de cada elemento de la matriz representa la amplitud de la onda en un instante de tiempo.Como el valor de Samplig es de 16000 Hz, esta matriz tendrá 16,000 elementos por segundo.
- `genre`: el género de la música como entero. Podemos usar el método `int2str()` del `feature` `genre()` para obtener el género en formato legible.

In [18]:
int2str = gtzan["train"].features["genre"].int2str
int2str(gtzan['train'][0]['genre'])

'pop'

## Prueba del modelo sin entrenamiento

Antes de comenzar a entrenar el modelo, probaremos el modelo sin entrenamiento para ver cómo se comporta. Usaremos el modelo `distilhubert`, un modelo previamente entrenado para clasificar el audio y fácil de refinar.
Para usar el modelo usaremos la clase `pipeline` de la librería Transformers.

In [6]:
from transformers import pipeline
import torch

classifier = pipeline(
    "audio-classification", model="ntu-spml/distilhubert",
    batch_size=16,
    device=torch.device('cuda')
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Device set to use cuda


In [7]:
classifier(gtzan['train'][0]['audio'])

[{'score': 0.522322952747345, 'label': 'LABEL_1'},
 {'score': 0.47767701745033264, 'label': 'LABEL_0'}]

Calcularemos la precisión del modelo sin entrenamiento. Para esto usaremos el conjunto de datos de prueba.

Lo primero que haremos es calcular las predicciones del modelo para cada muestra del conjunto de datos de prueba.

A continuación, mostraremos las predicciones del modelo para la primera muestra del conjunto de datos de prueba.

In [8]:
predictions = [classifier(sample['audio']) for sample in gtzan['test']]
predictions[0]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'score': 0.5148762464523315, 'label': 'LABEL_1'},
 {'score': 0.48512372374534607, 'label': 'LABEL_0'}]

Una vez que tengamos las predicciones del modelo, las compararemos con las etiquetas reales para calcular la precisión del modelo.

A continuación, mostraremos la precisión del modelo.

In [9]:
from sklearn.metrics import accuracy_score

y_true = [f"LABEL_{sample['genre']}" for sample in gtzan['test']]
y_pred = [prediction[0]['label'] for prediction in predictions]

accuracy_score(y_true, y_pred)

0.09


## Entrenamiento del modelo

Como podemos ver, el modelo sin entrenamiento tiene una precisión del 10%, muy poco. Esto se debe a que el modelo no ha sido entrenado con el conjunto de datos GTZAN.

Para entrenar el modelo usaremos la clase `Trainer` de la librería Transformers. Esta clase nos permite entrenar modelos de una manera simple y eficiente.

Mientras que con otros modelos necesitamos un `Tokenizer` en este caso usaremos un `feature_extractor`. Esta clase nos permitirá procesar muestras de audio para convertirlas en un formato que pueda procesar el modelo.

Entonces crearemos el `feature_extractor` que usaremos para entrenar el modelo.

In [10]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

A continuación, procesaremos las muestras de audio, lo que las convierte en un formato que puede procesar el modelo. En nuestro caso, reduciremos las muestras de audio a 30 segundos utilizando las opciones `max_length` y `padding` del `feature_extractor` y eliminaremos los datos que no estamos interesados ​​en el conjunto de datos con el método `remove_columns`.

In [11]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [12]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=16,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

Renombramos la columna `genre` a `label` para que el `Trainer` pueda identificarlo como una columna de etiquetas.

In [16]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

ValueError: Original column name genre not in the dataset. Current columns in the dataset: ['label', 'input_values', 'attention_mask']

Por último, antes de comenzar a entrenar el modelo, crearemos un diccionario en la correspondencia entre los nombres de los géneros y sus valores enteros, de modo que el `Trainer` pueda identificarlos y permitir un cambio rápido entre los dos formatos.

In [42]:
id2label = {
    str(i): int2str(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

## Entrenamiento del modelo

A continuación, crearemos el modelo que entrenaremos.

In [20]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained(
    model_id, num_labels=len(id2label)
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Entonces crearemos el `TrainerArguments`, que nos permitirá configurar el `Trainer` para entrenar el modelo.

In [21]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 3

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Entonces crearemos el `Trainer`, clase que estará a cargo de entrenar al modelo

In [22]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [23]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: martinez_dav (martinez_dav-ies-eduardo-primo-marques) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.814900,1.695361,0.520000
2,1.285700,1.294805,0.700000


Epoch,Training Loss,Validation Loss,Accuracy
1,1.814900,1.695361,0.520000
2,1.285700,1.294805,0.700000
3,1.343500,1.212400,0.680000


TrainOutput(global_step=339, training_loss=1.613198188905519, metrics={'train_runtime': 1992.46, 'train_samples_per_second': 1.354, 'train_steps_per_second': 0.17, 'total_flos': 1.8401964823872e+17, 'train_loss': 1.613198188905519, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 1.2948046922683716,
 'eval_accuracy': 0.7,
 'eval_runtime': 81.147,
 'eval_samples_per_second': 1.232,
 'eval_steps_per_second': 0.16,
 'epoch': 3.0}

## Uso del modelo entrenado

Lo primero que haremos es crear la `pipeline` con el modelo que hemos entrenado para clasificar los géneros musicales.

In [25]:
music_classifier = pipeline(
    "audio-classification",
    model=model,
    feature_extractor=feature_extractor,
    batch_size=16,
    device=torch.device('cuda')
)

Device set to use cuda


Como alternativa, podemos usar un modelo similar al nuestro, ya preentrenado

In [ ]:
music_classifier = pipeline(
    "audio-classification",
    model="ihanif/distilhubert-music-gtzan-classification",
    batch_size=16,
    device=torch.device('cuda')
)

config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/94.8M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Device set to use cuda


## Clasificar canciones

Una vez que se ha creado la tubería noy, podemos usarla para clasificar una canción.Este método recibe la ruta como un parámetro en la canción que queremos clasificar y devolver el género de la canción.

Necesitaremos música para poder clasificarla. Aquí hay algunos enlaces de música gratuitos para descargar y usar este cuaderno:

- [Música clásica](https://freemusicarchive.org/genre/Classical)
- [Música electrónica](https://freemusicarchive.org/genre/Electronic)
- [Música pop](https://freemusicarchive.org/genre/Pop)
- [Música rock](https://freemusicarchive.org/genre/Rock)
- [Música jazz](https://freemusicarchive.org/genre/Jazz)

También proporcionamos enlaces directos que a algunas `muestras` que puede descargar en la librería `requests`:

> ## IMPORTANTE
> Parece que el método descrito ha dejado de funcionar, así que lo que haremos será subir mp3's nuestros (guiño, guiño...) a los archivos de colab, y intentar identificarlos.


In [26]:
import requests

"""
#ONLINE NO FUNCIONA?
musica = [
    "https://archive.org/download/cd_the-very-best-of-frank-sinatra_frank-sinatra-frank-nancy-sinatra/disc1/01.07.%20Frank%20Sinatra%20-%20Fly%20Me%20to%20the%20Moon%20%28In%20Other%20Words%29_sample.mp3",
    "https://archive.org/download/cd_random-access-memories_daft-punk/disc1/05.%20Daft%20Punk%20-%20Instant%20Crush_sample.mp3",
    "https://ia800101.us.archive.org/21/items/cd_debbie-does-dallas_andrew-sherman/disc1/02.%20Andrew%20Sherman%20-%20Overture_sample.mp3",
    "https://archive.org/download/cd_greatest-hits_journey/Journey%20%281988%29%20-%20Journey%27s%20Greatest%20Hits%20%5BFLAC%5D/02%20-%20Don%27t%20Stop%20Believin%27_sample.mp3",
    "https://archive.org/download/cd_generator_bad-religion/disc1/01.%20Bad%20Religion%20-%20Generator_sample.mp3",
    "https://archive.org/download/geniesduclassique_vol1no12/1-09%20Concerto%20Brandeburghese%20No.3%20-%20Adagio.mp3"
]

filenames = []

for url in musica:
    filename = url.split("/")[-1]
    filenames.append(filename)

    with open(filename, "wb") as f:
        f.write(requests.get(url).content)
"""

#LOCAL
filenames = [
    "BADJOVI.mp3",
    "BEETHOVEN.mp3",
    "CHRISTMAS.mp3",
    "FITOCASA.mp3",
    "MOZART.mp3",
    "PERSONALJESUS.mp3",
    "SHAKIRAHIPS.mp3"
]

filenames



['BADJOVI.mp3',
 'BEETHOVEN.mp3',
 'CHRISTMAS.mp3',
 'FITOCASA.mp3',
 'MOZART.mp3',
 'PERSONALJESUS.mp3',
 'SHAKIRAHIPS.mp3']

In [44]:
import numpy as np
for filename in filenames:
    clasificacion = music_classifier(filename)
    etiqueta=id2label[clasificacion[0]['label'].split('_')[-1]]
    print(f"Classificació de {filename}: {etiqueta} {clasificacion}")


Classificació de BADJOVI.mp3: metal [{'score': 0.4537466764450073, 'label': 'LABEL_6'}, {'score': 0.1697184145450592, 'label': 'LABEL_9'}, {'score': 0.10369613766670227, 'label': 'LABEL_3'}, {'score': 0.07683499157428741, 'label': 'LABEL_2'}, {'score': 0.05309874564409256, 'label': 'LABEL_1'}]
Classificació de BEETHOVEN.mp3: classical [{'score': 0.45514991879463196, 'label': 'LABEL_1'}, {'score': 0.1605546921491623, 'label': 'LABEL_5'}, {'score': 0.0945928543806076, 'label': 'LABEL_2'}, {'score': 0.08784794062376022, 'label': 'LABEL_9'}, {'score': 0.06317416578531265, 'label': 'LABEL_0'}]
Classificació de CHRISTMAS.mp3: pop [{'score': 0.32268232107162476, 'label': 'LABEL_7'}, {'score': 0.13294686377048492, 'label': 'LABEL_9'}, {'score': 0.12810371816158295, 'label': 'LABEL_2'}, {'score': 0.09889425337314606, 'label': 'LABEL_3'}, {'score': 0.09138411283493042, 'label': 'LABEL_1'}]
Classificació de FITOCASA.mp3: blues [{'score': 0.20916691422462463, 'label': 'LABEL_0'}, {'score': 0.16914